In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANTITATIVE BETTING PORTFOLIO (DYNAMIC)         ")
print("=======================================================")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
BANKROLL = 30.00        
MIN_EDGE = 0.03         # 3% Edge required to bet
MAX_ODDS_CAP = 6.00     # Strict Cap: We don't bet on miracles (>6.00)
BASE_KELLY = 0.20       # Use 20% of Full Kelly (Professional Standard)

# --- FIXTURES ---
fixtures = [
    {"Home": "Manchester United", "Away": "Newcastle United", "Odds_1": 2.53, "Odds_X": 3.61, "Odds_2": 2.68},
    {"Home": "Nottingham Forest", "Away": "Manchester City", "Odds_1": 5.29, "Odds_X": 4.45, "Odds_2": 1.58},
    {"Home": "Arsenal",           "Away": "Brighton & Hove Albion", "Odds_1": 1.40, "Odds_X": 4.92, "Odds_2": 7.85},
    {"Home": "Brentford",         "Away": "Bournemouth",      "Odds_1": 2.29, "Odds_X": 3.60, "Odds_2": 3.03},
    {"Home": "Burnley",           "Away": "Everton",          "Odds_1": 4.01, "Odds_X": 3.35, "Odds_2": 2.01},
    {"Home": "Liverpool",         "Away": "Wolverhampton Wanderers", "Odds_1": 1.24, "Odds_X": 6.59, "Odds_2": 11.28},
    {"Home": "West Ham United",   "Away": "Fulham",           "Odds_1": 2.68, "Odds_X": 3.41, "Odds_2": 2.63},
    {"Home": "Chelsea",           "Away": "Aston Villa",      "Odds_1": 1.85, "Odds_X": 3.88, "Odds_2": 4.03},
    {"Home": "Sunderland",        "Away": "Leeds United",     "Odds_1": 2.58, "Odds_X": 3.23, "Odds_2": 2.86},
    {"Home": "Crystal Palace",    "Away": "Tottenham Hotspur", "Odds_1": 2.29, "Odds_X": 3.30, "Odds_2": 3.25},
]

# ==============================================================================
# 2. LOAD MODEL
# ==============================================================================
try:
    artifacts = joblib.load('football_model_final.pkl')
    model = artifacts['model']
    features = artifacts['features']
    current_elos = artifacts['elo_dict']
    df_recent = artifacts['df_recent']
    print(f"✅ Model Loaded. Features: {len(features)}")
except:
    print("❌ Error loading pickle.")
    exit()

def get_stats(team):
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    
    stats = {}
    roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points', 'fouls', 'team_score']
    for f in roll_feats:
        col = f"{prefix}roll_{f}"
        stats[f] = last[col] if col in last else 0
    return stats

# ==============================================================================
# 3. ANALYSIS ENGINE
# ==============================================================================
opportunities = []

print(f"\n🔍 ANALYZING {len(fixtures)} MATCHES...")

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    h_stats, a_stats = get_stats(h_team), get_stats(a_team)
    h_elo, a_elo = current_elos.get(h_team, 1500), current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats: continue

    # --- BUILD FEATURES ---
    input_data = {
        'diff_elo': (h_elo + 60) - a_elo, 
        'home_elo': h_elo,
        'away_elo': a_elo,
        'diff_rest': 0, 
        'home_roll_team_xg': h_stats['team_xg'],
        'away_roll_team_xg': a_stats['team_xg'],
        'home_roll_team_possession': h_stats['team_possession'],
        'away_roll_team_possession': a_stats['team_possession'],
        'home_roll_shots_onTarget': h_stats['shots_onTarget'],
        'away_roll_shots_onTarget': a_stats['shots_onTarget'],
        'home_roll_corners': h_stats['corners'],
        'away_roll_corners': a_stats['corners'],
        'home_roll_fouls': h_stats['fouls'],
        'away_roll_fouls': a_stats['fouls']
    }
    
    # Diffs
    roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points', 'fouls']
    for feat in roll_feats:
        input_data[f"diff_{feat}"] = h_stats[feat] - a_stats[feat]

    # Predict
    input_df = pd.DataFrame([input_data]).reindex(columns=features, fill_value=0)
    probs = model.predict_proba(input_df)[0]
    p_away, p_draw, p_home = probs[0], probs[1], probs[2]

    # Print Raw Probabilities
    print(f"{h_team} vs {a_team}: H {p_home:.2f} | D {p_draw:.2f} | A {p_away:.2f}")

    # --- VALUE FINDER (Pure Math, No Team Bias) ---
    bets_for_match = []
    
    # Check all 3 outcomes
    for outcome, p, odds, label in [
        ('HOME', p_home, f['Odds_1'], h_team),
        ('DRAW', p_draw, f['Odds_X'], 'Draw'),
        ('AWAY', p_away, f['Odds_2'], a_team)
    ]:
        # 1. Odds Cap (Risk Management)
        if odds >= MAX_ODDS_CAP: continue
            
        # 2. Edge Calculation
        edge = (p * odds) - 1
        
        # 3. Dynamic Thresholds
        # Draw needs slightly more probability to be worth the variance
        min_prob = 
        
        if edge > MIN_EDGE and p > min_prob:
            bets_for_match.append({
                'Match': f"{h_team} vs {a_team}",
                'Type': outcome,
                'Team': label,
                'Odds': odds,
                'Prob': p,
                'Edge': edge
            })

    # Pick BEST bet per match
    if bets_for_match:
        best = sorted(bets_for_match, key=lambda x: x['Edge'], reverse=True)[0]
        opportunities.append(best)

# ==============================================================================
# 4. PORTFOLIO ALLOCATION (KELLY)
# ==============================================================================
if opportunities:
    res = pd.DataFrame(opportunities)
    
    def calculate_stake(row):
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        f = ((b * p) - q) / b
        
        # DYNAMIC STAKING: 
        # If Odds are high (e.g. 5.00), reduce multiplier to limit volatility
        # If Odds are low (e.g. 1.80), allow full multiplier
        volatility_dampener = 1.0
        if row['Odds'] > 3.0: volatility_dampener = 0.8
        if row['Odds'] > 4.5: volatility_dampener = 0.6
        
        # Apply Base Kelly * Dampener
        stake_pct = f * BASE_KELLY * volatility_dampener
        
        # Hard Cap at 8% of Bankroll
        return max(0, min(stake_pct, 0.08))

    res['Stake_Pct'] = res.apply(calculate_stake, axis=1)
    res['Stake'] = res['Stake_Pct'] * BANKROLL
    res['Profit'] = res['Stake'] * (res['Odds'] - 1)
    
    res = res[res['Stake'] > 0.01].sort_values('Stake', ascending=False)
    
    # --- JUPYTER DISPLAY ---
    display_df = res[['Match', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Stake', 'Profit']].copy()
    display_df.columns = ['Match', 'Bet Type', 'Selection', 'Odds', 'Win %', 'Edge', 'Stake ($)', 'Profit ($)']
    
    styled = (display_df.style
        .format({'Odds': '{:.2f}', 'Win %': '{:.1%}', 'Edge': '{:.1%}', 'Stake ($)': '${:.2f}', 'Profit ($)': '${:.2f}'})
        .background_gradient(subset=['Win %'], cmap='Greens', vmin=0.2, vmax=0.8)
        .background_gradient(subset=['Edge'], cmap='Blues', vmin=0.0, vmax=0.5)
        .bar(subset=['Stake ($)'], color='#ff6b6b', vmin=0)
        .bar(subset=['Profit ($)'], color='#51cf66', vmin=0)
        .set_caption(f"🚀 PURE QUANT BETTING SLIP (No Hard Filters) | Bankroll: ${BANKROLL}")
    )
    display(styled)
    
    print(f"\n💰 Total Risk: ${res['Stake'].sum():.2f}")
    print(f"💵 Remaining:  ${BANKROLL - res['Stake'].sum():.2f}")
else:
    print("\n📉 The model found no value bets this week.")

   🤖 QUANTITATIVE BETTING PORTFOLIO (DYNAMIC)         
✅ Model Loaded. Features: 18

🔍 ANALYZING 10 MATCHES...
Manchester United vs Newcastle United: H 0.54 | D 0.21 | A 0.26
Nottingham Forest vs Manchester City: H 0.23 | D 0.22 | A 0.55
Arsenal vs Brighton & Hove Albion: H 0.60 | D 0.22 | A 0.18
Brentford vs Bournemouth: H 0.41 | D 0.25 | A 0.34
Burnley vs Everton: H 0.39 | D 0.22 | A 0.39
Liverpool vs Wolverhampton Wanderers: H 0.76 | D 0.14 | A 0.10
West Ham United vs Fulham: H 0.44 | D 0.21 | A 0.35
Chelsea vs Aston Villa: H 0.39 | D 0.22 | A 0.39
Sunderland vs Leeds United: H 0.48 | D 0.25 | A 0.27
Crystal Palace vs Tottenham Hotspur: H 0.52 | D 0.23 | A 0.25


,Match,Bet Type,Selection,Odds,Win %,Edge,Stake ($),Profit ($)
0,Manchester United vs Newcastle United,HOME,Manchester United,2.53,53.7%,35.9%,$1.41,$2.15
4,Chelsea vs Aston Villa,AWAY,Aston Villa,4.03,39.3%,58.3%,$0.92,$2.80
5,Sunderland vs Leeds United,HOME,Sunderland,2.58,48.0%,23.9%,$0.91,$1.43
6,Crystal Palace vs Tottenham Hotspur,HOME,Crystal Palace,2.29,52.2%,19.4%,$0.90,$1.17
2,Burnley vs Everton,HOME,Burnley,4.01,39.0%,56.3%,$0.90,$2.70
3,West Ham United vs Fulham,HOME,West Ham United,2.68,44.4%,19.0%,$0.68,$1.14
1,Nottingham Forest vs Manchester City,HOME,Nottingham Forest,5.29,23.5%,24.1%,$0.20,$0.87



💰 Total Risk: $5.92
💵 Remaining:  $24.08


In [55]:
# ==========================================
# MATCH EXPLAINER (Why did the model bet?)
# ==========================================

def explain_match(home_team, away_team):
    print(f"🕵️ DETECTIVE MODE: {home_team} vs {away_team}")
    
    # 1. Get Data (Reusing logic from engine)
    h_stats = get_stats(home_team)
    a_stats = get_stats(away_team)
    h_elo = current_elos.get(home_team, 1500)
    a_elo = current_elos.get(away_team, 1500)
    
    if not h_stats or not a_stats:
        print("❌ Data missing for one of the teams.")
        return

    # 2. Build Comparison Table
    comparison_data = []
    
    # A. ELO Rating
    # Add Home Adv (+65) to Home ELO to show "Effective ELO"
    effective_h_elo = h_elo + 65
    elo_diff = effective_h_elo - a_elo
    adv = home_team if elo_diff > 0 else away_team
    
    comparison_data.append({
        'Feature': 'ELO Rating (Power)', 
        f'{home_team}': int(h_elo), 
        f'{away_team}': int(a_elo), 
        'Difference': f"{int(elo_diff)}",
        'Advantage': adv
    })
    
    # B. Rolling Stats
    # Key stats the model cares about
    keys = {
        'team_points': 'Form (Avg Points)',
        'team_xg': 'Attack (Avg xG)',
        'team_possession': 'Control (Possession)',
        'shots_onTarget': 'Danger (Shots on Target)',
        'corners': 'Pressure (Corners)',
        'fouls': 'Aggression (Fouls)'
    }
    
    for k, label in keys.items():
        h_val = h_stats.get(k, 0)
        a_val = a_stats.get(k, 0)
        diff = h_val - a_val
        
        # Determine who is better
        # For Fouls, usually "Less" is not necessarily better or worse, but let's assume higher stats = advantage for others
        if k == 'fouls':
            adv_team = "-"
        else:
            adv_team = home_team if h_val > a_val else away_team
            
        comparison_data.append({
            'Feature': label, 
            f'{home_team}': round(h_val, 2), 
            f'{away_team}': round(a_val, 2), 
            'Difference': round(diff, 2),
            'Advantage': adv_team
        })

    # 3. Create DataFrame
    comp_df = pd.DataFrame(comparison_data)
    
    # 4. Display with Styling
    def highlight_advantage(row):
        # Color the advantage column green/red logic could go here
        # For simplicity, we bold the advantage
        return ['' for _ in row]

    styler = (comp_df.style
        .set_caption(f"📊 STATISTICAL FACE-OFF: {home_team} vs {away_team}")
        .hide(axis="index")
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#404040'), ('color', 'white'), ('font-weight', 'bold')]},
            {'selector': 'td', 'props': [('text-align', 'center')]},
        ])
        .background_gradient(subset=['Difference'], cmap='coolwarm', vmin=-1, vmax=1)
    )
    
    display(styler)
    
    # 5. NARRATIVE EXPLANATION
    print("\n🧠 MODEL REASONING:")
    
    # Check ELO
    if abs(elo_diff) < 50:
        print(f"1. ELO: Teams are rated very similarly (Diff {int(elo_diff)}).")
    elif elo_diff > 50:
        print(f"1. ELO: {home_team} is significantly stronger on paper.")
    else:
        print(f"1. ELO: {away_team} is rated higher despite playing away.")
        
    # Check Form (Points)
    p_diff = h_stats['team_points'] - a_stats['team_points']
    if p_diff < -0.5:
        print(f"2. FORM: {away_team} is in much better form ({a_stats['team_points']:.1f} pts/game vs {h_stats['team_points']:.1f}).")
        print(f"   -> This is likely why the model picked the Away win.")
    elif p_diff > 0.5:
        print(f"2. FORM: {home_team} is dominating recent games.")
    else:
        print(f"2. FORM: Both teams have similar recent results.")
        
    # Check xG
    xg_diff = h_stats['team_xg'] - a_stats['team_xg']
    if xg_diff < -0.3:
        print(f"3. PERFORMANCE: {away_team} is creating far better chances (xG gap: {xg_diff:.2f}).")
        print("   -> Even if they lost recently, the model sees they are playing well (Underlying Stats).")
    print("\n\n")

In [19]:
# ==========================================
# TEST IT ON YOUR BETS
# ==========================================
# Explain the specific bet you were curious about
for fixture in fixtures:
    explain_match(fixture['Home'], fixture['Away'])

🕵️ DETECTIVE MODE: Manchester United vs Newcastle United


Feature,Manchester United,Newcastle United,Difference,Advantage
ELO Rating (Power),1537.000000,1559.000000,43,Manchester United
Form (Avg Points),1.600000,1.600000,0.000000,Newcastle United
Attack (Avg xG),2.420000,1.520000,0.900000,Manchester United
Control (Possession),0.620000,0.550000,0.070000,Manchester United
Danger (Shots on Target),6.200000,4.800000,1.400000,Manchester United
Pressure (Corners),6.600000,7.600000,-1.000000,Newcastle United
Aggression (Fouls),13.200000,8.200000,5.000000,-



🧠 MODEL REASONING:
1. ELO: Teams are rated very similarly (Diff 43).
2. FORM: Both teams have similar recent results.



🕵️ DETECTIVE MODE: Nottingham Forest vs Manchester City


Feature,Nottingham Forest,Manchester City,Difference,Advantage
ELO Rating (Power),1487.000000,1618.000000,-65,Manchester City
Form (Avg Points),1.800000,3.000000,-1.200000,Manchester City
Attack (Avg xG),1.200000,2.200000,-1.000000,Manchester City
Control (Possession),0.530000,0.550000,-0.020000,Manchester City
Danger (Shots on Target),4.800000,5.600000,-0.800000,Manchester City
Pressure (Corners),5.800000,3.600000,2.200000,Nottingham Forest
Aggression (Fouls),12.000000,9.400000,2.600000,-



🧠 MODEL REASONING:
1. ELO: Manchester City is rated higher despite playing away.
2. FORM: Manchester City is in much better form (3.0 pts/game vs 1.8).
   -> This is likely why the model picked the Away win.
3. PERFORMANCE: Manchester City is creating far better chances (xG gap: -1.00).
   -> Even if they lost recently, the model sees they are playing well (Underlying Stats).



🕵️ DETECTIVE MODE: Arsenal vs Brighton & Hove Albion


...


🧠 MODEL REASONING:
1. ELO: Teams are rated very similarly (Diff 17).
2. FORM: Aston Villa is in much better form (3.0 pts/game vs 1.4).
   -> This is likely why the model picked the Away win.



🕵️ DETECTIVE MODE: Sunderland vs Leeds United


Feature,Sunderland,Leeds United,Difference,Advantage
ELO Rating (Power),1512.000000,1467.000000,109,Sunderland
Form (Avg Points),1.400000,1.000000,0.400000,Sunderland
Attack (Avg xG),0.620000,1.700000,-1.080000,Leeds United
Control (Possession),0.400000,0.410000,-0.010000,Leeds United
Danger (Shots on Target),2.800000,3.800000,-1.000000,Leeds United
Pressure (Corners),4.200000,3.200000,1.000000,Sunderland
Aggression (Fouls),10.400000,12.400000,-2.000000,-



🧠 MODEL REASONING:
1. ELO: Sunderland is significantly stronger on paper.
2. FORM: Both teams have similar recent results.
3. PERFORMANCE: Leeds United is creating far better chances (xG gap: -1.08).
   -> Even if they lost recently, the model sees they are playing well (Underlying Stats).



🕵️ DETECTIVE MODE: Crystal Palace vs Tottenham Hotspur


Feature,Crystal Palace,Tottenham Hotspur,Difference,Advantage
ELO Rating (Power),1564.000000,1542.000000,86,Crystal Palace
Form (Avg Points),1.400000,1.400000,0.000000,Tottenham Hotspur
Attack (Avg xG),1.980000,1.360000,0.620000,Crystal Palace
Control (Possession),0.480000,0.530000,-0.050000,Tottenham Hotspur
Danger (Shots on Target),5.400000,3.800000,1.600000,Crystal Palace
Pressure (Corners),6.000000,6.800000,-0.800000,Tottenham Hotspur
Aggression (Fouls),10.400000,11.200000,-0.800000,-



🧠 MODEL REASONING:
1. ELO: Crystal Palace is significantly stronger on paper.
2. FORM: Both teams have similar recent results.



